<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/C%C3%B3pia_de_modelos_tradicionais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [363]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/Jefferson - Dados3 - Copia.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

excluir setor financeiro e empresas com PL negativo, numeros faltante e transformar eventuais caracteres em números

In [364]:
# Agora, vamos remover as linhas onde 'd9' seja igual a 1
dados = dados[dados['d11'] != 1]
# Converte a coluna 'PL2022' para números (float) e remove as linhas onde 'PL2022' seja negativo
dados['PL2022'] = pd.to_numeric(dados['PL2022'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN
dados = dados[dados['PL2022'] > 0]

# Agora, vamos remover as linhas onde 'VM30Abril11' seja igual a '-'
dados = dados[dados['VM30Abril2023'] != '-']
dados = dados[dados['PL2022'] != '-']
dados = dados[dados['LL2022'] != '-']
dados = dados[dados['AT2021'] != '-']
dados = dados[dados['AT2021'] != '0']

#transforma as variáveis em números
dados['AT2021'] = pd.to_numeric(dados['AT2021'], errors='coerce')
dados['LL2022'] = pd.to_numeric(dados['LL2022'], errors='coerce')
dados['VM30Abril2023'] = pd.to_numeric(dados['VM30Abril2023'], errors='coerce')
dados['PL2022'] = pd.to_numeric(dados['PL2022'], errors='coerce')

# Lista de variáveis que você deseja escalonar
variaveis = ['VM30Abril2023', 'PL2022', 'LL2022', 'AT2021']

# Verifique o tipo de dados de cada coluna
for coluna in variaveis:
    print(f"Tipo de dados da coluna {coluna}: {dados[coluna].dtype}")


Tipo de dados da coluna VM30Abril2023: float64
Tipo de dados da coluna PL2022: float64
Tipo de dados da coluna LL2022: float64
Tipo de dados da coluna AT2021: float64


ESCALONAMENTO DAS VARIÁVEIS

In [375]:
variaveis = ['VM30Abril2023', 'PL2022', 'LL2022']

for coluna in variaveis:
    dados[coluna] = dados[coluna] / dados['AT2021']


WINSORIZAÇÃO: tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [365]:
import pandas as pd
import numpy as np
from scipy.stats import mstats

# Supondo que seu DataFrame se chame "dados"

# Lista de variáveis para winsorizar
variaveis_winsorizar = ['VM30Abril2023', 'PL2022', 'LL2022', 'AT2021']

# Converter as colunas para números, tratando não numéricos como NaN
for coluna in variaveis_winsorizar:
    dados[coluna] = pd.to_numeric(dados[coluna], errors='coerce')

# Aplicar winsorização de 1% nas variáveis
for coluna in variaveis_winsorizar:
    dados[coluna] = mstats.winsorize(dados[coluna], limits=[0.01, 0.01])

# Agora, as variáveis especificadas foram winsorizadas


MQO_SIMPLES

In [376]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_white

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Visualize os resultados do modelo
print(modelo.summary())

# Calcule o MedAE
y_pred = modelo.predict(X)
medae = np.median(np.abs(y - y_pred))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)

# Calcular os VIFs (Fator de Inflação da Variância)
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data["Variável"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data

vif_result = calculate_vif(X.drop(columns=['const']))
print("\nFator de Inflação da Variância (VIF):")
print(vif_result)

# Realizar o teste de heterocedasticidade de White
white_test_result = het_white(modelo.resid, X)
print("\nResultado do teste de heterocedasticidade de White:")
print("LM Statistic:", white_test_result[0])
print("LM p-value:", white_test_result[1])
print("F Statistic:", white_test_result[2])
print("F p-value:", white_test_result[3])


                            OLS Regression Results                            
Dep. Variable:          VM30Abril2023   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     33.96
Date:                Fri, 29 Sep 2023   Prob (F-statistic):           9.86e-14
Time:                        18:14:50   Log-Likelihood:                -248.23
No. Observations:                 246   AIC:                             502.5
Df Residuals:                     243   BIC:                             513.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1333      0.091      1.460      0.1

MQP - MODELO SIMPLES

In [367]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear robusta usando o método WLS com pesos robustos
modelo_robusto = sm.RLM(y, X, M=sm.robust.norms.HuberT()).fit()

# Visualize os resultados do modelo robusto
print(modelo_robusto.summary())

# Calcule o MedAE
y_pred_robusto = modelo_robusto.predict(X)
medae_robusto = np.median(np.abs(y - y_pred_robusto))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled_robusto = (medae_robusto / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) robusto:", medae_robusto)
print("MedAE robusto escalado pelo valor (ou preço) em percentagem:", medae_scaled_robusto)

# Calcular os VIFs (Fator de Inflação da Variância)
def calculate_vif(dataframe):
    vif_data = pd.DataFrame()
    vif_data["Variável"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) for i in range(dataframe.shape[1])]
    return vif_data

vif_result = calculate_vif(X)
print("\nFator de Inflação da Variância (VIF):")
print(vif_result)

# Calcular o coeficiente de determinação (R-squared)
sst_robusto = ((y - y.mean())**2).sum()
ssr_robusto = ((y - y_pred_robusto)**2).sum()
r_squared_robusto = 1 - (ssr_robusto / sst_robusto)

print("Coeficiente de Determinação (R-squared) do modelo robusto:", r_squared_robusto)

# Realizar o teste de heterocedasticidade de White
white_test_result_robusto = het_white(modelo_robusto.resid, X)
print("\nResultado do teste de heterocedasticidade de White para o modelo robusto:")
print("LM Statistic:", white_test_result_robusto[0])
print("LM p-value:", white_test_result_robusto[1])
print("F Statistic:", white_test_result_robusto[2])
print("F p-value:", white_test_result_robusto[3])


                    Robust linear Model Regression Results                    
Dep. Variable:          VM30Abril2023   No. Observations:                  246
Model:                            RLM   Df Residuals:                      243
Method:                          IRLS   Df Model:                            2
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 29 Sep 2023                                         
Time:                        18:10:47                                         
No. Iterations:                     2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        3.43e+04   1.59e+05      0.216      0.8

REGRESSÃO QUANTÍLICA_SIMPLES (NÃO LINEAR)

In [368]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão quantílica
modelo = sm.QuantReg(y, X).fit(q=0.5)  # q=0.5 representa a mediana, ajuste conforme necessário

# Visualize os resultados do modelo
print(modelo.summary())
# Faça previsões com o modelo quantílico
y_pred_quantreg = modelo.predict(X)

# Calcule o MedAE para o modelo quantílico
medae_quantreg = np.median(np.abs(y - y_pred_quantreg))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço) em percentagem
medae_scaled_quantreg = (medae_quantreg / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) quantílico:", medae_quantreg)
print("MedAE quantílico escalado pelo valor (ou preço) em percentagem:", medae_scaled_quantreg)


                         QuantReg Regression Results                          
Dep. Variable:          VM30Abril2023   Pseudo R-squared:               0.5085
Model:                       QuantReg   Bandwidth:                   1.287e+06
Method:                 Least Squares   Sparsity:                    3.939e+06
Date:                Fri, 29 Sep 2023   No. Observations:                  246
Time:                        18:11:25   Df Residuals:                      243
                                        Df Model:                            2
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.061e-06   1.37e+05   2.96e-11      1.000   -2.71e+05    2.71e+05
PL2022         1.0052      0.013     80.109      0.000       0.980       1.030
LL2022         2.7034      0.062     43.797      0.000       2.582       2.825

The condition number is large, 1.84e+07. This might

RB_Simples (LINEAR)

In [369]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo Bayesian Ridge
    modelo_bayesiano = BayesianRidge()

    modelo_bayesiano.fit(X_train, y_train)
    y_pred = modelo_bayesiano.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = median_absolute_error(y_test, y_pred)
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar os coeficientes do modelo (coeficientes Bayesianos)
    coeficientes = modelo_bayesiano.coef_
    print("Coeficientes do Modelo:", coeficientes)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Coeficientes do Modelo: [1.09496416 3.0827152 ]
Coeficientes do Modelo: [1.10378178 3.0041204 ]
Coeficientes do Modelo: [1.11672089 2.8167814 ]
Coeficientes do Modelo: [1.06880059 2.74790512]
Coeficientes do Modelo: [1.16092566 2.98783373]
Coeficientes do Modelo: [1.28990021 1.89259576]
Coeficientes do Modelo: [1.06748728 2.73815894]
Coeficientes do Modelo: [1.15174708 2.85102453]
Coeficientes do Modelo: [1.10945519 2.9942111 ]
Coeficientes do Modelo: [0.76367139 5.90357852]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 12.80729568829546
Média do R²: 0.3851663766667889


SVR (NÃO LINEAR)

In [370]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo SVR com kernel RBF
    #modelo_svr = SVR(kernel='linear', C=1.0)
    modelo_svr = SVR(kernel='poly', degree=3, C=1.0)
    #modelo_svr = SVR(kernel='sigmoid', C=1.0)
    #modelo_svr = SVR(kernel='rbf', C=1.0)

    modelo_svr.fit(X_train, y_train)
    y_pred = modelo_svr.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = median_absolute_error(y_test, y_pred)
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Média do MedAE escalado pelo valor (ou preço) em percentagem: 25.534220304658284
Média do R²: -0.13871688818620667


CART_simples (NÃO LINEAR)

In [371]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de árvore de decisão com os parâmetros desejados
    modelo_arvore = DecisionTreeRegressor(min_samples_split=5, random_state=42)
    #modelo_arvore = DecisionTreeRegressor(max_depth=5, min_samples_split=5, random_state=42)



    # Criar o modelo Bagging com 500 árvores
    bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=500, bootstrap=True, random_state=42)
    #bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=100, bootstrap=True, random_state=42)


    bagging_model.fit(X_train, y_train)
    y_pred = bagging_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância dos coeficientes (importância das variáveis)
    importancia_coeficientes = np.mean([tree.feature_importances_ for tree in bagging_model.estimators_], axis=0)
    print("Importância dos Coeficientes:", importancia_coeficientes)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [0.63708518 0.36291482]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [0.71281737 0.28718263]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


...

Importância dos Coeficientes: [0.68372232 0.31627768]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [0.70696797 0.29303203]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [0.30597649 0.69402351]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 16.195079881263496
Média do R²: 0.208577300580827


FLORESTA ALEATÓRIA (NÃO LINEAR)

In [372]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Floresta Aleatória com os parâmetros desejados
    random_forest_model = RandomForestRegressor(n_estimators=500, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_features=4, random_state=42)
    #random_forest_model = RandomForestRegressor(n_estimators=500, max_features=0.3, random_state=42)


    random_forest_model.fit(X_train, y_train)
    y_pred = random_forest_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância das variáveis
    importancia_variaveis = random_forest_model.feature_importances_
    print("Importância das Variáveis:", importancia_variaveis)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Importância das Variáveis: [0.6014478 0.3985522]
Importância das Variáveis: [0.68101544 0.31898456]
Importância das Variáveis: [0.75112975 0.24887025]
Importância das Variáveis: [0.65669327 0.34330673]
Importância das Variáveis: [0.64979467 0.35020533]
Importância das Variáveis: [0.67760129 0.32239871]
Importância das Variáveis: [0.61434064 0.38565936]
Importância das Variáveis: [0.6501698 0.3498302]
Importância das Variáveis: [0.67157866 0.32842134]
Importância das Variáveis: [0.31332753 0.68667247]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 16.11599070483846
Média do R²: 0.2925546511549239


GRADIENT BOOSTING (NÃO LINEAR)

In [373]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Regressão Gradient Boosting com os parâmetros desejados
    gradient_boosting_model = GradientBoostingRegressor(n_estimators=500, random_state=42)

    gradient_boosting_model.fit(X_train, y_train)
    y_pred = gradient_boosting_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância das variáveis
    importancia_variaveis = gradient_boosting_model.feature_importances_
    print("Importância das Variáveis:", importancia_variaveis)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Importância das Variáveis: [0.82775244 0.17224756]
Importância das Variáveis: [0.85649199 0.14350801]
Importância das Variáveis: [0.8608571 0.1391429]
Importância das Variáveis: [0.82153161 0.17846839]
Importância das Variáveis: [0.85733797 0.14266203]
Importância das Variáveis: [0.90893471 0.09106529]
Importância das Variáveis: [0.81785091 0.18214909]
Importância das Variáveis: [0.86235823 0.13764177]
Importância das Variáveis: [0.85844044 0.14155956]
Importância das Variáveis: [0.33120555 0.66879445]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 12.945683962266822
Média do R²: 0.34078517604474695


REDES NEURAIS (NÃO LINEAR)

In [374]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2022', 'LL2022']]
y = dados['VM30Abril2023']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de regressão de rede neural com os parâmetros desejados
    modelo_rede_neural = MLPRegressor(hidden_layer_sizes=(30), max_iter=1000, random_state=42)

    modelo_rede_neural.fit(X_train, y_train)
    y_pred = modelo_rede_neural.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Média do MedAE escalado pelo valor (ou preço) em percentagem: 10.99601525356775
Média do R²: 0.47740790999762817
